# We're scraping

Probably going to just scrape it all. 

See [baseball-scraper](https://pypi.org/project/baseball-scraper/) for examples.

In [1]:
from baseball_scraper import statcast
from baseball_scraper import playerid_lookup

import pandas as pd 
import datetime #don't know if I'll need this.

In [2]:
##What happens if I ask for a date on which no games are played... 
##Like I write a function and uh-oh, all-star break included...

##Looking for a date pre statcast gives a table with lots of NaN values. 
#pre_stat_scrape = statcast(start_dt='2010-08-08', verbose=True)

##Looking for a date on which no games were played gives a dataframe of shape 0x90
#january_scrape = statcast(start_dt='2018-01-01', verbose=True)

start_data = datetime.datetime(2018,1,1)
pd.to_datetime(start_data).date().strftime("%Y-%m-%d")



'2018-01-01'

In [3]:
start = datetime.date(2020,7,23).strftime("%Y-%m-%d")
end = datetime.date(2020,8,18).strftime("%Y-%m-%d")##This is the world's third hardest way to write '2020-08-18', but you're maybe using it later.

sc_2020 = statcast(start,end)

This is a large query, it may take a moment to complete
Completed sub-query from 2020-07-23 to 2020-07-28
Completed sub-query from 2020-07-29 to 2020-08-03
Completed sub-query from 2020-08-04 to 2020-08-09
Completed sub-query from 2020-08-10 to 2020-08-15
Completed sub-query from 2020-08-16 to 2020-08-18


In [5]:
sc_2020.to_csv('statcast2020.csv')

In [6]:
del sc_2020

In [7]:
print(datetime.datetime.now())
#start = datetime.date(2019,3,20).strftime("%Y-%m-%d")
#end = datetime.date(2019,10,1).strftime("%Y-%m-%d")
###sc_2019 = statcast(start,end)## THis ended up producing an error.
###I guess I'm not surprised. There's a lot going on here.datetime
def big_scrape(start,end):
    scrapes = [] ##
    while True:
        try:
            temp = statcast(start.strftime("%Y-%m-%d"), (start+datetime.timedelta(days=4)).strftime("%Y-%m-%d"))
        except ParserError:
            print("ParserError failure at {}".format(start.strftime("%Y-%m-%d")))
            break
        except:
            print("Unspecified failure at {}".format(start.strftime("%Y-%m-%d")))
            break
        scrapes.append(temp)
        start = start+datetime.timedelta(days=5)
        if start > end:
            break
    return scrapes


print(datetime.datetime.now())

2020-08-19 10:32:58.162340
2020-08-19 10:32:58.162670


In [8]:
for dt in [2019,2018,2017,2016]:
    start = datetime.date(dt,3,20)
    end = datetime.date(dt,10,1)
    scrape = big_scrape(start,end)
    pd.concat(scrape).to_csv('statcast'+str(dt)+'.csv')
    print("Finished with statcast year {}.".format(dt))
    del scrape

NameError: name 'df' is not defined

,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,...,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,154,SI,2019-03-29,100.2,-2.549981,6.274024,Jordan Hicks,606115.0,663855.0,field_out,...,5.0,9.0,5.0,9.0,9.0,5.0,5.0,9.0,Strategic,Standard
1,155,SL,2019-03-29,84.2,-2.557062,6.021211,Jordan Hicks,606115.0,663855.0,NaN,...,5.0,9.0,5.0,9.0,9.0,5.0,5.0,9.0,Strategic,Standard
2,158,CH,2019-03-29,89.4,-2.563425,5.978502,Jordan Hicks,518735.0,663855.0,strikeout,...,5.0,9.0,5.0,9.0,9.0,5.0,5.0,9.0,Infield shift,Standard
3,159,SI,2019-03-29,100.7,-2.313411,6.069608,Jordan Hicks,518735.0,663855.0,NaN,...,5.0,9.0,5.0,9.0,9.0,5.0,5.0,9.0,Infield shift,Standard
4,162,SI,2019-03-29,100.1,-2.392935,6.314507,Jordan Hicks,518735.0,663855.0,NaN,...,5.0,9.0,5.0,9.0,9.0,5.0,5.0,9.0,Infield shift,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6536,6477,CH,2019-03-28,88.0,-2.868703,4.934759,Luis Castillo,466320.0,622491.0,strikeout,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Strategic,Standard
6537,6484,FF,2019-03-28,96.4,-2.773471,5.243966,Luis Castillo,466320.0,622491.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Strategic,Standard
6538,6509,CH,2019-03-28,88.6,-2.876455,5.095479,Luis Castillo,466320.0,622491.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Strategic,Standard
6539,6511,FF,2019-03-28,95.8,-2.762156,5.190272,Luis Castillo,466320.0,622491.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Strategic,Standard
